In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gpt2writedata/data.json
/kaggle/input/gpt2writedata/train.json
/kaggle/input/gpt2writedata/train_clear.txt
/kaggle/input/gpt2writedata/train.txt
/kaggle/input/gpt2writedata/article.db
/kaggle/input/gpt2writedata/train_clear_web_mini.txt


# 预处理数据

In [2]:
!git clone https://github.com/napoler/reformer-chinese-pytorch
%cd reformer-chinese-pytorch
!mkdir data
!head -n 1200 /kaggle/input/gpt2writedata/train_clear_web_mini.txt > ./data/data.txt

Cloning into 'reformer-chinese-pytorch'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 66 (delta 38), reused 42 (delta 20), pack-reused 0
Unpacking objects: 100% (66/66), done.
/kaggle/working/reformer-chinese-pytorch


In [3]:
!python bulidData.py

Downloading: 100%|████████████████████████████| 110k/110k [00:00<00:00, 660kB/s]
Downloading: 100%|███████████████████████████| 29.0/29.0 [00:00<00:00, 21.2kB/s]
Downloading: 100%|███████████████████████████| 269k/269k [00:00<00:00, 1.06MB/s]
1200it [00:00, 351330.78it/s]


In [4]:
# !head -n 1000 /kaggle/input/gpt2writedata/train_clear_web_mini.txt >test.txt

# 安装依赖

In [5]:
# !pip install -r requirements.txt

!pip install pytorch_lightning
!pip install wandb -q
!pip install tkitJson
!pip install jsonargparse[signatures]
!pip install transformers reformer_pytorch==1.4.3 pytorch_lightning==1.4.8 tqdm==4.62.3 tkitLr==0.0.0.216325705 

  Created wheel for tkitJson: filename=tkitJson-0.0.0.3-py3-none-any.whl size=2696 sha256=645422c68639f4ebc9c16989450051c6f517f0e8e4335e80dc6cb8a07f0bb534
  Stored in directory: /root/.cache/pip/wheels/16/e9/fb/69e56e98874a965c5d16b29d23a2702ba31364150087221a3c
Successfully built tkitJson
     |████████████████████████████████| 127 kB 515 kB/s 
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ done
    Preparing wheel metadata ... - \ done
  Created wheel for docstring-parser: filename=docstring_parser-0.13-py3-none-any.whl size=31866 sha256=03a85418d0c0be3b233465b24dd0835979370295c25e1f2b59b50033d4034ba9
  Stored in directory: /root/.cache/pip/wheels/bd/88/3c/d1aa049309f7945178cac9fbe6561a86424f432da57c18ca0f
Successfully built docstring-parser
     |████████████████████████████████| 924 kB 511 kB/s 
     |████████████████████████████████| 76 kB 3.7 MB/s 
  Created wheel for axial-positional-embedding: filename=axial_pos

# wandb 设置

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

os.environ["WANDB_SILENT"] = "true"
os.environ["WANDB_API_KEY"] = secret_value_0

# 重写生成训练参数

In [7]:
%%writefile my_config_test_cuda_16g.yaml

seed_everything: 288
trainer:
  logger:
  - class_path: pytorch_lightning.loggers.WandbLogger
    init_args:
      log_model: false
      offline: false
      prefix: ''
      project: "litGPT"
  checkpoint_callback: true
  callbacks:
  - class_path: pytorch_lightning.callbacks.EarlyStopping
    init_args:
      check_finite: true
      check_on_train_epoch_end: false
      min_delta: 0.0
      mode: min
      monitor: val_loss
      patience: 100
      strict: true
      verbose: true
  - class_path: pytorch_lightning.callbacks.LearningRateMonitor
    init_args:
      log_momentum: false
      logging_interval: step
  - class_path: pytorch_lightning.callbacks.ModelCheckpoint
    init_args:
      auto_insert_metric_name: true
      filename: '{epoch}-{val_loss:.2f}-{val_acc:.2f}'
      mode: min
      monitor: val_loss
      save_last: true
      save_top_k: 2
      save_weights_only: false
      verbose: true
  default_root_dir: null
  gradient_clip_val: 0.0
  gradient_clip_algorithm: norm
  process_position: 0
  num_nodes: 1
  num_processes: 1
  devices: null
  gpus: 1
  auto_select_gpus: false
  tpu_cores: null
  ipus: null
  log_gpu_memory: null
  progress_bar_refresh_rate: null
  overfit_batches: 0.0
  track_grad_norm: -1
  check_val_every_n_epoch: 1
  fast_dev_run: false
  accumulate_grad_batches: 1
  max_epochs: 10
  min_epochs: null
  max_steps: null
  min_steps: null
  max_time: null
  limit_train_batches: 1000
  limit_val_batches: 200
  limit_test_batches: 200
  limit_predict_batches: 1.0
  val_check_interval: 1.0
  flush_logs_every_n_steps: 100
  log_every_n_steps: 50
  accelerator: null
  sync_batchnorm: false
  precision: 16
  weights_summary: top
  weights_save_path: null
  num_sanity_val_steps: 2
  truncated_bptt_steps: null
  resume_from_checkpoint: null
  profiler: null
  benchmark: false
  deterministic: false
  reload_dataloaders_every_n_epochs: 0
  reload_dataloaders_every_epoch: false
  auto_lr_find: false
  replace_sampler_ddp: true
  terminate_on_nan: false
  auto_scale_batch_size: false
  prepare_data_per_node: true
  plugins: null
  amp_backend: native
  amp_level: O2
  distributed_backend: null
  move_metrics_to_cpu: false
  multiple_trainloader_mode: max_size_cycle
  stochastic_weight_avg: false
model:
  dim: 128
  depth: 6
  max_seq_len: 512
  lsh_dropout: 0.1
  optimizer_name: AdamW
  learning_rate: 0.0001
  full_attn_thres: 128
  from_pretrained: bert-base-chinese
  batch_size: 64
  trainfile: ./data/train.pkt
  valfile: ./data/val.pkt
  testfile: ./data/test.pkt


Writing my_config_test_cuda_16g.yaml


# 开始训练

In [8]:
!python trainer.py --config my_config_test_cuda_16g.yaml

"batch_size":      64
"depth":           6
"dim":             128
"from_pretrained": bert-base-chinese
"full_attn_thres": 128
"learning_rate":   0.0001
"lsh_dropout":     0.1
"max_seq_len":     512
"optimizer_name":  AdamW
"testfile":        ./data/test.pkt
"trainfile":       ./data/train.pkt
"valfile":         ./data/val.pkt
2022-02-24 04:49:15.548045: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Epoch 0:   0%|                                 | 0/17 [00:00<00:00, 2516.08it/s]
CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.






























































